# Optimally Creating and Assigning Work Orders Based on Routes

Suppose our organization needs to perform restaurant/brewery inspections in the Greater Portland, Maine area. Let's assume that there are around 25 breweries that need to be inspected and that there are 5 workers that are available to do the inspections. As the supervisor of these workers I'm going to develop a Python Script (well, Jupyter Notebook in this case) that will optimally create distinct routes for my workers, create assignments at the brewery locations, and then assign the assignment to the correct worker.

### Import ArcGIS API for Python
Let's import some libraries and connect to our organization

In [ ]:
import pandas as pd
import arcgis
from arcgis.gis import GIS
from arcgis.apps import workforce
pd.options.display.max_columns = None

gis = GIS("https://arcgis.com", "workforce_scripts")
project = workforce.Project(gis.content.search("type:'Workforce Project' Maine Brewery Inspections")[0])
project.assignments_item.layers[0].delete_features(where="1=1")

### View the breweries that need to be inspected

In [ ]:
breweries = gis.content.search("type:'Feature Service' owner:workforce_scripts Maine Breweries")[0].layers[0]
breweries.filter = "location in ('Portland','South Portland','Gorham','Biddeford','Scarborough', 'Topsham','Freeport')"
webmap = gis.map("Portland, ME", zoomlevel=10)
webmap.add_layer(breweries)
webmap

In [ ]:
breweries_df = breweries.query(where=breweries.filter, out_fields="objectid,name,location,url", as_df=True)
breweries_df

### Create a route for each worker

Now that we know what we're working with, let's use the Plan Routes tool to generate the most optimal routes for each of the workers. First we need to define where the workers will start their routes. Each worker will begin from the main office located at 100 Commercial Street, Portland Maine. We'll use the geocoding module to get an exact location for this address.

In [ ]:
from arcgis.geocoding import geocode
start_location = geocode("100 Commercial Street, Portland, ME", out_sr={"wkid": 102100})[0]["location"]
start_location["spatialReference"] = {"wkid": 102100}
start_location

Next we need to create a feature at this location

In [ ]:
feature = arcgis.features.Feature(
    attributes={
        "ObjectID": 1,
        "Name": "Office"
    },
    geometry=start_location
)

Next, we'll create a Feature Set from the feature. Then we'll create a Feature Collection from the Feature Set. Finally, we'll format the layer so that it conforms to the expected input format defined [here](https://doc.arcgis.com/en/arcgis-online/analyze/plan-routes.htm).

In [ ]:
feature_set = arcgis.features.FeatureSet([feature])
feature_collection = arcgis.features.FeatureCollection.from_featureset(feature_set)
start_layer = {"layerDefinition": feature_collection.properties["layers"][0]["layerDefinition"], "featureSet": feature_set.value}

Then we'll run the Plan Routes tool using the breweries layer as list of stops to route to. We'll set the number of routes equal to the number of workers. We'll also set the start time and start location as well as few other parameters.

In [ ]:
from datetime import datetime
workers = project.workers.search()
results = arcgis.features.analysis.plan_routes(breweries,       # Feature Layer of Stops
                                    len(workers),               # Number of routes to generate
                                    5,                          # Maximum stops per route
                                    datetime.now(),             # Start time of route
                                    start_layer,                # The dictionary we created to represent the start location
                                    stop_service_time=60,       # How much time in minutes to spend at each stop
                                    max_route_time=480,         # The maximum time for the worker to complete the route
                                    )
results

Let's see what the routes look like

In [ ]:
webmap = gis.map("Portland, ME", zoomlevel=10)
webmap.add_layer(results["routes_layer"])
webmap.add_layer(results["assigned_stops_layer"])
webmap

Let's look at what data is in route

In [ ]:
routes = results['routes_layer'].query().sdf
routes

You can see that each route has a name, total time, and total distance among other things. Let's see what information is provided in an assigned stop.

In [ ]:
stops = results['assigned_stops_layer'].query().sdf
stops

You can see each row in the above table contains the attributes of each Brewery along with information about which route it is on. You'll also notice that there are several additional stops not related to a brewery. These are the starting and ending locations of each route.

### Create Assignment and Assign To Worker

For each route that was generated we will select a random worker to complete that route. Then we'll find the breweries that were assigned to that route and create an Inspection Assignment for each one. Notice that when the assignment is created we are also assigning it to a worker.

An important thing to note is that we are setting the due date of the assignment to the departure date of the stop. This means that a mobile worker will be able to sort their "To Do" list by due date and see the assignments in the correct order (according to the route).

In [ ]:
import random

assignments_to_add = []
for _, row in routes.iterrows():
    worker = random.choice(workers)
    workers.remove(worker)
    route_stops = stops.loc[(stops['RouteName'] == row["RouteName"]) & stops['globalid'].notnull()]
    for _, stop in route_stops.iterrows():
        assignments_to_add.append(workforce.Assignment(
            project,
            assignment_type="Inspection",
            location=stop["name"],
            status="assigned",
            worker=worker,
            assigned_date=datetime.now(),
            due_date=stop["DepartTime"],
            geometry=stop["SHAPE"]
        ))
assignments = project.assignments.batch_add(assignments_to_add)

Let's check to verify the assignments were created and are assigned

In [ ]:
webmap.add_layer(project.assignments_layer)
webmap